In [1]:
import requests
import json
import pandas as pd
from processing_split import processing_split

In [2]:
# Ссылка на страницу словаря
def get_link(page):
    return f'https://api.mansitranslator.ru/api/v1/mansi_words/?page={page}&search='

# Количество страниц (выявлено в ручную)
page_limit = 88

# Словари для слов
list_lang_mans = []
list_lang_rus = []
list_lang_comments = []

In [3]:
# Парсинг каждой страницы
print("Парсинг успешно начат!")
for page in range(1, page_limit):
    print(f">>> Парсинг {page} страницы:")
    response = requests.get(url=get_link(page))
    for i in range(len(json.loads(response.text)["results"])):
        # Выбрать response по порядковому номеру
        result = json.loads(response.text)["results"][i]

        # Достать слово на мансийском языке и его перевод на русском
        word_mans = result["value"].replace(' (сыгвинский диалект)', '')
        try:
            word_rus_trash = result["rus_words"][0]["value"]
            words_rus_and_comments = processing_split(word_rus_trash)
        except IndexError:
            word_rus_trash = None
            words_rus_and_comments = [[None, None]]

        # Сохранение копий перевода
        for group in words_rus_and_comments:
            list_lang_mans.append(word_mans)
            list_lang_rus.append(group[0])
            list_lang_comments.append(group[1])

            print(f"    Добавлена пара-слов: {word_mans} — {group[0]} ({group[1]})")
print("Парсинг успешно завершён!")

Парсинг успешно начат!
>>> Парсинг 1 страницы:
    Добавлена пара-слов: аквъёт э̄ргын ма̄хум — хор (None)
    Добавлена пара-слов: агтуӈкве — верить (None)
    Добавлена пара-слов: агтуӈкве — поверить (None)
    Добавлена пара-слов: аквъюипа̄лт — подряд (None)
    Добавлена пара-слов: аквъюипа̄лт — друг за другом (None)
    Добавлена пара-слов: аквъяныт — равный (по размеру)
    Добавлена пара-слов: аки — дед (None)
    Добавлена пара-слов: акван-юрысьхатуӈкве — объединиться (None)
    Добавлена пара-слов: алгаль — вверх (по реке)
    Добавлена пара-слов: алн — в верховье реки (None)
    Добавлена пара-слов: алпи — тело (None)
    Добавлена пара-слов: алпӣсов — кожа (человека)
    Добавлена пара-слов: алпӣхар — поверхность тела человека (None)
    Добавлена пара-слов: агтыл — убеждение (None)
    Добавлена пара-слов: агылтта̄нтуӈкве — уверять (None)
    Добавлена пара-слов: аигла̄луӈкве — пить (много раз)
    Добавлена пара-слов: айтуӈкве — поить (None)
    Добавлена пара-слов: айтуӈкв

In [4]:
# Создание DataFrame
df = pd.DataFrame(
    {'target': list_lang_mans,
    'source': list_lang_rus,
    'comment': list_lang_comments}
    )

,target,source,comment
0,аквъёт э̄ргын ма̄хум,хор,None
1,агтуӈкве,верить,None
2,агтуӈкве,поверить,None
3,аквъюипа̄лт,подряд,None
4,аквъюипа̄лт,друг за другом,None
...,...,...,...
6249,хулуӈкве,берёсте,None
6250,хӯлыглаӈкве,слышать,None
6251,хӯнтамла̄лыглаӈкве,прислушиваться,None
6252,хӯрыгсов,мешочек,None


In [5]:
# Сохранение DataFrame to File
df.to_csv("dataset_dict1.csv", sep="|")